<a href="https://colab.research.google.com/github/haeseokoh/00_exchange/blob/main/%EC%83%81%EB%8C%80%EA%B0%95%EB%8F%84_%EC%A2%85%EB%AA%A9(%ED%95%9C%EA%B5%AD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ----------------------------
# 1. 라이브러리 설치
# ----------------------------
!pip install numpy==1.24.3 --quiet
!pip install --force-reinstall yfinance beautifulsoup4 lxml openpyxl --quiet

# ----------------------------
# 2. 라이브러리 임포트
# ----------------------------
import yfinance as yf
import pandas as pd
import numpy as np
from google.colab import files
import io

# ----------------------------
# 3. 엑셀 파일 업로드 및 티커 처리
# ----------------------------
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df_tickers = pd.read_excel(io.BytesIO(uploaded[filename]))

df_tickers['Ticker'] = df_tickers.iloc[:, 0].astype(str).str.lstrip('A')
df_tickers['Suffix'] = df_tickers.iloc[:, 2].astype(str).str.upper().apply(lambda x: '.KS' if x == 'KS' else '.KQ' if x == 'KQ' else '')
df_tickers['YahooTicker'] = df_tickers['Ticker'] + df_tickers['Suffix']

company_map = pd.Series(df_tickers.iloc[:, 1].values, index=df_tickers['YahooTicker']).to_dict()
tickers = df_tickers['YahooTicker'].tolist()

# ----------------------------
# 4. 벤치마크 다운로드 (KOSPI: ^KS11)
# ----------------------------
kospi = yf.download("^KS11", period="1y")["Close"]

# ----------------------------
# 5. Mansfield RS 계산 함수
# ----------------------------
def compute_mansfield_rs(price_series, benchmark_series, ma_period=52):
    relative = price_series / benchmark_series
    ma = relative.rolling(window=ma_period).mean()
    return ((relative / ma) - 1) * 100

def normalize_to_100_scale(x, scale=12):
    return 100 * (1 / (1 + np.exp(-x / scale)))

# ----------------------------
# 6. RS 계산 실행
# ----------------------------
mansfield_results = []
all_rs_results = []

for ticker in tickers:
    try:
        price = yf.download(ticker, period="1y")["Close"]
        aligned = pd.concat([price, kospi], axis=1, join='inner')
        aligned.columns = ['stock', 'kospi']
        rs_series = compute_mansfield_rs(aligned['stock'], aligned['kospi']).dropna()
        if not rs_series.empty:
            latest_rs_raw = rs_series.iloc[-1]
            normalized_rs = normalize_to_100_scale(latest_rs_raw)
            all_rs_results.append((ticker, round(normalized_rs, 2), round(latest_rs_raw, 2)))
            if normalized_rs >= 70:
                mansfield_results.append((ticker, round(normalized_rs, 2), round(latest_rs_raw, 2)))
    except:
        continue

# ----------------------------
# 7. RS 결과 출력 (그래프 제거 버전)
# ----------------------------
if mansfield_results:
    mansfield_df = pd.DataFrame(mansfield_results, columns=["Ticker", "Normalized_RS", "Raw_RS"])
    mansfield_df["Ticker (RS)"] = mansfield_df["Ticker"].map(company_map)
    mansfield_df = mansfield_df.sort_values(by="Normalized_RS", ascending=False)

    print("📌 정규화된 Mansfield RS ≥ 70 종목 (vs KOSPI 기준, 0~100 스케일):")
    display(mansfield_df[["Ticker (RS)", "Normalized_RS", "Raw_RS"]])

else:
    print("🚫 RS ≥ 70 조건을 만족하는 종목이 없습니다. → RS 상위 10개 종목 출력:")
    fallback_df = pd.DataFrame(all_rs_results, columns=["Ticker", "Normalized_RS", "Raw_RS"])
    fallback_df["Name"] = fallback_df["Ticker"].map(company_map)
    fallback_df = fallback_df.sort_values(by="Normalized_RS", ascending=False).head(10)
    display(fallback_df)

# ----------------------------
# 8. 모멘텀 상위 20 출력 (회사명 기준)
# ----------------------------
print("\n📌 변동성 조정 모멘텀 상위 20 (엑셀 내 종목명 기준):")

data = yf.download(tickers, period="1y")['Close']
data = data.dropna(axis=1, how='any').sort_index()

def calculate_risk_adjusted_momentum(data, period_3m=63, period_6m=126, period_12m=252, last_n_days=10):
    daily_top_20_stocks = {}
    trading_days = data.index[-last_n_days:]

    for day in trading_days:
        end_idx = data.index.get_loc(day)
        start_3m = max(0, end_idx - period_3m + 1)
        start_6m = max(0, end_idx - period_6m + 1)
        start_12m = max(0, end_idx - period_12m + 1)

        returns_3m = data.iloc[start_3m:end_idx + 1].pct_change().dropna().mean()
        returns_6m = data.iloc[start_6m:end_idx + 1].pct_change().dropna().mean()
        returns_12m = data.iloc[start_12m:end_idx + 1].pct_change().dropna().mean()

        vol_3m = data.iloc[start_3m:end_idx + 1].pct_change().dropna().std()
        vol_6m = data.iloc[start_6m:end_idx + 1].pct_change().dropna().std()
        vol_12m = data.iloc[start_12m:end_idx + 1].pct_change().dropna().std()

        risk_adj_3m = returns_3m / vol_3m
        risk_adj_6m = returns_6m / vol_6m
        risk_adj_12m = returns_12m / vol_12m

        avg_risk_adj_momentum = (risk_adj_3m + risk_adj_6m + risk_adj_12m) / 3
        top_20 = avg_risk_adj_momentum.nlargest(20).index.tolist()
        daily_top_20_stocks[day] = top_20

    df = pd.DataFrame.from_dict(daily_top_20_stocks, orient='index', columns=[f'Top {i+1}' for i in range(20)])
    df.index.name = 'Date'
    return df

momentum_df = calculate_risk_adjusted_momentum(data, last_n_days=10)

# 종목명을 기반으로 출력
momentum_df_named = momentum_df.copy()
for col in momentum_df_named.columns:
    momentum_df_named[col] = momentum_df_named[col].map(company_map)

display(momentum_df_named)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 30.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
blosc2 3.3.1 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, 

Saving 상대강도계산.xlsx to 상대강도계산.xlsx
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^KS11']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['2023-12-15']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['2023-12-18']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['2023-12-19']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['2023-12-20']: YFRateLimitError('Too Many Requests. Rate 

🚫 RS ≥ 70 조건을 만족하는 종목이 없습니다. → RS 상위 10개 종목 출력:


,Ticker,Normalized_RS,Raw_RS,Name



📌 변동성 조정 모멘텀 상위 20 (엑셀 내 종목명 기준):


[*********************100%***********************]  333 of 333 completed
ERROR:yfinance:
333 Failed downloads:
ERROR:yfinance:['2024-04-30', '2024-05-16', '2024-06-04', '2025-03-17', '2024-12-13', '2024-06-19', '2024-01-23', '2024-03-28', '2024-03-18', '2024-02-23', '2025-04-07', '2024-11-28', '2025-04-30', '2024-05-07', '2024-07-16', '2024-01-16', '2024-08-01', '2024-10-21', '2025-04-22', '2024-08-06', '2024-10-17', '2024-12-20', '2025-01-22', '2024-01-19', '2024-10-10', '2024-01-09', '2024-10-08', '2024-05-28', '2024-08-14', '2024-06-05', '2024-07-12', '2024-11-04', '2024-01-05', '2024-11-19', '2024-06-14', '2025-01-08', '2025-02-19', '2024-08-02', '2025-01-21', '2024-11-25', '2024-07-17', '2024-12-03', '2024-03-25', '2024-04-29', '2024-03-22', '2024-04-19', '2024-03-11', '2024-01-22', '2024-01-03', '2024-11-13', '2023-12-15', '2024-11-22', '2024-01-10', '2024-08-28', '2025-04-29', '2025-02-10', '2025-04-02', '2023-12-19', '2024-01-25', '2024-03-15', '2025-03-06', '2024-06-18', '2024

,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10,Top 11,Top 12,Top 13,Top 14,Top 15,Top 16,Top 17,Top 18,Top 19,Top 20
Date,,,,,,,,,,,,,,,,,,,,
